# Medoid Compositing
*Seasonal Composite Landsat TM/ETM+ Images Using the Medoid (a Multi-Dimensional Median), Neil Flood, 2013, doi:10.3390/rs5126481*

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import tools, composite, cloud_mask, indices

In [3]:
from ipygee import *

## Build a collection

In [4]:
p = ee.Geometry.Point(-72, -42)

In [5]:
col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
        .filterBounds(p).filterDate('2017-01-01', '2017-12-01')\
        .map(cloud_mask.landsat8SRPixelQA())\
        .map(lambda img: img.addBands(indices.ndvi(img,'B5', 'B4')))\
        .limit(7)

In [6]:
eprint(col.size())

## Other simple composites to compare

In [7]:
max_ndvi = col.qualityMosaic('ndvi')

In [8]:
mosaic = col.mosaic()

## Medoid

In [9]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

### add date band before compositing

In [10]:
def add_date(img):
    date = tools.date.getDateBand(img)
    return img.addBands(date).copyProperties(date, ['day_since_epoch'])
col = col.map(add_date)

In [11]:
minval = ee.Number(col.aggregate_min('day_since_epoch'))

In [12]:
maxval = ee.Number(col.aggregate_max('day_since_epoch'))

In [13]:
medoid = composite.medoid(col, bands=bands)

In [14]:
medoid = medoid.set('min_date', minval, 'max_date', maxval)

## Medoid without taking in count zero values

In [15]:
medoid_no_zeros = composite.medoid(col, bands=bands, discard_zeros=True)

In [16]:
medoid_no_zeros = medoid_no_zeros.set('min_date', minval, 'max_date', maxval)

## Show on Map

In [17]:
Map = Map()
Map.show()

Map(basemap={'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a…

In [18]:
vis = {'bands':['B5', 'B6','B4'], 'min':0, 'max':5000}

In [19]:
Map.addLayer(p)
Map.centerObject(p)

In [20]:
Map.addLayer(max_ndvi, vis, 'max NDVI')

In [21]:
Map.addLayer(mosaic, vis, 'simply Mosaic')

In [22]:
Map.addLayer(medoid, vis, 'Medoid')

In [23]:
Map.addLayer(medoid.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates')

In [24]:
Map.addLayer(medoid_no_zeros, vis, 'Medoid without zero values')

In [25]:
Map.addLayer(medoid_no_zeros.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates of medoid without zero values')

## Extract data from images and compute locally to compare

Extract medoid values in point

In [27]:
medoid_values = tools.image.getValue(medoid.select(bands), p, scale=30, side='client')

In [28]:
medoid_values

{'B2': 77, 'B3': 188, 'B4': 105, 'B5': 2500, 'B6': 691, 'B7': 224}

List of values

In [29]:
medoid_values_list = [val for _, val in medoid_values.items()]

In [30]:
medoid_values_list

[224, 2500, 77, 691, 188, 105]

Extract values at point in each image of the collection

In [31]:
col_values = tools.imagecollection.getValues(col.select(bands), p, scale=30, side='client')

Get bandnames

In [32]:
col_key_list = []
for _, d in col_values.items():
    keys = []
    for k, v in d.items():
        keys.append(k)        
    col_key_list.append(keys)

In [33]:
col_key_list

[['B7', 'B5', 'B2', 'B6', 'B3', 'B4'],
 ['B7', 'B5', 'B2', 'B6', 'B3', 'B4'],
 ['B7', 'B5', 'B2', 'B6', 'B3', 'B4'],
 ['B7', 'B5', 'B2', 'B6', 'B3', 'B4'],
 ['B7', 'B5', 'B2', 'B6', 'B3', 'B4'],
 ['B7', 'B5', 'B2', 'B6', 'B3', 'B4'],
 ['B7', 'B5', 'B2', 'B6', 'B3', 'B4']]

Get values as a list

In [34]:
col_values_list = []
for _, d in col_values.items():
    values = []
    for _, v in d.items():
        if v:
            values.append(v)
        else:
            values.append(0)
    col_values_list.append(values)

In [35]:
col_values_list

[[224.0, 2445.0, 83.0, 698.0, 200.0, 127.0],
 [287.0, 3168.0, 112.0, 870.0, 272.0, 143.0],
 [259.0, 2717.0, 90.0, 813.0, 210.0, 120.0],
 [224.0, 2500.0, 77.0, 691.0, 188.0, 105.0],
 [245.0, 2465.0, 87.0, 720.0, 193.0, 107.0],
 [0, 0, 0, 0, 0, 0],
 [307.0, 3142.0, 107.0, 928.0, 290.0, 159.0]]

## Medoid Method locally

In [36]:
def local_medoid(values):
    from copy import copy
    import math

    def distance(arr1, arr2):
        zipped = zip(arr1, arr2)
        accum = 0
        for a, b in zipped:
            calc = (a-b)*(a-b)
            accum += calc
        return math.sqrt(accum)

    def med(values):
        results = {}
        for i, val in enumerate(values):
            val = list(val)
            cop = copy(values)
            cop = [list(a) for a in cop]
            cop.remove(val)
            dist = 0
            for r in cop:
                r = list(r)
                d = distance(val, r)
                dist += d
            results[i] = dist

        return results
    
    def getmin(d):
        minval = min(d.values())
        for k, v in d.items():
            if v == minval:
                return k
    
    values = med(values)
    min_value = getmin(values)
    
    # return the index of the minimized sum as first argument, and all options as second
    return min_value, values

## Compute medoid locally and compare

In [37]:
local = local_medoid(col_values_list)

In [38]:
local

(3,
 {0: 4461.227704323096,
  1: 6022.895734074118,
  2: 4593.00466857401,
  3: 4380.0310171598885,
  4: 4399.422196724632,
  5: 17251.127189606195,
  6: 5996.407209928899})

Get the values that correspond to the medoid

In [39]:
min_values = col_values_list[local[0]]

In [40]:
min_values

[224.0, 2500.0, 77.0, 691.0, 188.0, 105.0]

Match bands with values

In [41]:
local_medoid = dict(zip(col_key_list[0], min_values))

In [42]:
local_medoid

{'B2': 77.0, 'B3': 188.0, 'B4': 105.0, 'B5': 2500.0, 'B6': 691.0, 'B7': 224.0}

In [43]:
medoid_values

{'B2': 77, 'B3': 188, 'B4': 105, 'B5': 2500, 'B6': 691, 'B7': 224}

## Finally, compare values from medoid mosaic against locally computed medoid (from images values)

In [44]:
medoid_values == local_medoid

True